In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os

os.environ['GORDON_DATA_DIR']='/oasis/projects/nsf/csd181/yuncong/DavidData2014tif'
os.environ['GORDON_REPO_DIR']='/oasis/projects/nsf/csd395/yuncong/Brain'
os.environ['GORDON_RESULT_DIR']='/oasis/projects/nsf/csd395/yuncong/DavidData2014results'

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

import utilities2014
reload(utilities2014)
from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [ ]:
# Over-segment the image into superpixels using SLIC (http://ivrg.epfl.ch/research/superpixels)

from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_ubyte, pad
from joblib import Parallel, delayed

masked_image = dm.image.copy()

# approx_bg_intensity = dm.image[10:20, 10:20].mean()
# masked_image[~dm.mask] = approx_bg_intensity

masked_image[~dm.mask] = 0

segmentation = slic(gray2rgb(masked_image), n_segments=int(dm.segm_params['n_superpixels']), 
                    max_iter=10, 
                    compactness=float(dm.segm_params['slic_compactness']), 
                    sigma=float(dm.segm_params['slic_sigma']), 
                    enforce_connectivity=True)

In [5]:
# display(mark_boundaries(masked_image, segmentation))

In [6]:
n = len(np.unique(segmentation))

def f(i):
    m = dm.mask[segmentation == i]
    return np.count_nonzero(m)/float(len(m)) < .8

q = Parallel(n_jobs=16)(delayed(f)(i) for i in range(n))
                        
masked_segmentation = segmentation.copy()
for i in np.where(q)[0]:
    masked_segmentation[segmentation == i] = -1

In [7]:
# display(mark_boundaries(masked_image, masked_segmentation))

In [8]:
from skimage.segmentation import relabel_sequential

# try:
#     masked_segmentation_relabeled = dm.load_pipeline_result('segmentation', 'npy')

# except:

    # segmentation starts from 0
masked_segmentation_relabeled, fw, inv = relabel_sequential(masked_segmentation + 1)

# make background label -1
masked_segmentation_relabeled -= 1


In [9]:
dm.save_pipeline_result(masked_segmentation_relabeled, 'segmentation', 'npy')

saved /home/yuncong/project/DavidData2014results/RS141/0008/RS141_x5_0008_segm-blueNisslRegular_segmentation.npy


In [4]:
# masked_segmentation_relabeled = dm.load_pipeline_result('segmentation', 'npy')

In [11]:
sp_props = regionprops(masked_segmentation_relabeled + 1, intensity_image=dm.image, cache=True)

def obtain_props_worker(i):
    return sp_props[i].centroid, sp_props[i].area, sp_props[i].mean_intensity, sp_props[i].bbox

r = Parallel(n_jobs=16)(delayed(obtain_props_worker)(i) for i in range(len(sp_props)))

sp_centroids = np.array([s[0] for s in r])
sp_areas = np.array([s[1] for s in r])
sp_mean_intensity = np.array([s[2] for s in r])
sp_bbox = np.array([s[3] for s in r]) 

# n_superpixel x 8: (cy, cx, area, mean_intensity, ymin, xmin, ymax, xmax)

sp_properties = np.column_stack([sp_centroids, sp_areas, sp_mean_intensity, sp_bbox])

dm.save_pipeline_result(sp_properties, 'spProps', 'npy')

saved /home/yuncong/project/DavidData2014results/RS141/0008/RS141_x5_0008_segm-blueNisslRegular_spProps.npy


In [5]:
# sp_properties = dm.load_pipeline_result('spProps', 'npy')

In [6]:
n_superpixels = len(np.unique(masked_segmentation_relabeled)) - 1

In [12]:
img_superpixelized = mark_boundaries(dm.image, masked_segmentation_relabeled)
img_superpixelized_with_text = img_as_ubyte(img_superpixelized)

dm.save_pipeline_result(img_superpixelized_with_text, 'segmentationWithoutText', 'jpg')

for s in range(n_superpixels):
    cv2.putText(img_superpixelized_with_text, str(s), 
                           tuple(np.floor(sp_centroids[s][::-1]).astype(np.int) - np.array([10,-10])), 
                           cv2.FONT_HERSHEY_DUPLEX,
                           .5, ((255,0,255)), 1)

dm.save_pipeline_result(img_superpixelized_with_text, 'segmentationWithText', 'jpg')

saved /home/yuncong/project/DavidData2014results/RS141/0008/RS141_x5_0008_segm-blueNisslRegular_segmentationWithoutText.jpg
saved /home/yuncong/project/DavidData2014results/RS141/0008/RS141_x5_0008_segm-blueNisslRegular_segmentationWithText.jpg


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:107: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [13]:
emptycanvas_superpixelized = mark_boundaries(np.ones_like(dm.image), masked_segmentation_relabeled, 
                                             color=(0,0,0), outline_color=None)
# emptycanvas_superpixelized = img_as_ubyte(emptycanvas_superpixelized)
alpha_channel = ~ emptycanvas_superpixelized.all(axis=2)
a = np.dstack([emptycanvas_superpixelized, alpha_channel])

dm.save_pipeline_result(a, 'segmentationTransparent', 'png', is_rgb=True)

saved /home/yuncong/project/DavidData2014results/RS141/0008/RS141_x5_0008_segm-blueNisslRegular_segmentationTransparent.png


In [14]:
# display(img_superpixelized_with_text)

In [7]:
def diff_offset(x_offset, y_offset):

    h, w = masked_segmentation_relabeled.shape

    moving_x_low = max(x_offset, 0)
    moving_x_high = min(x_offset + w, w)
    moving_width = moving_x_high - moving_x_low
    moving_y_low = max(y_offset, 0)
    moving_y_high = min(y_offset + h, h)
    moving_height = moving_y_high - moving_y_low

    d = np.dstack([masked_segmentation_relabeled[moving_y_low:moving_y_high, moving_x_low:moving_x_high], 
                   masked_segmentation_relabeled[:moving_height, :moving_width]])

    ys, xs = np.mgrid[:d.shape[0], :d.shape[1]]
    nzs = ~(d[...,0]==d[...,1])
    r = np.c_[d[nzs], ys[nzs] + y_offset, xs[nzs] + x_offset] # [sp_label1, sp_label2, y, x]

    return r

In [9]:
from collections import defaultdict

t = time.time()

diffs = np.vstack([diff_offset(1,0), diff_offset(0,1), diff_offset(1,1), diff_offset(1,-1)])

edge_coords = defaultdict(set)
neighbors = [set() for _ in range(n_superpixels)]

for i, j, y, x in diffs:
    edge_coords[frozenset([i,j])].add((x,y))
    if i != -1:
        neighbors[i].add(j)
    if j != -1:
        neighbors[j].add(i)

print time.time() - t

# check symmetry; note that this CANNOT check if neighbors is complete
A = np.zeros((n_superpixels, n_superpixels))
for i, nbrs in enumerate(neighbors):
    q = list([j for j in nbrs if j != -1])
    A[i, q] = 1    
assert np.all(A == A.T), 'neighbor list is not symmetric'

dm.save_pipeline_result(neighbors, 'neighbors', 'pkl')

6.52944087982
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_segm-blueNisslRegular_neighbors.pkl


In [86]:
# from collections import defaultdict

# t = time.time()

# d1 = np.diff(masked_segmentation_relabeled, axis=0).astype(np.bool)
# d2 = np.diff(masked_segmentation_relabeled, axis=1).astype(np.bool)
# d3 = (masked_segmentation_relabeled[1:,1:]-masked_segmentation_relabeled[:-1,:-1]).astype(np.bool)
# d4 = (masked_segmentation_relabeled[1:,:-1]-masked_segmentation_relabeled[:-1,1:]).astype(np.bool)
# d5 = (masked_segmentation_relabeled[2:,:]-masked_segmentation_relabeled[:-2,:]).astype(np.bool)
# d6 = (masked_segmentation_relabeled[:,2:]-masked_segmentation_relabeled[:,:-2]).astype(np.bool)
# d7 = (masked_segmentation_relabeled[2:,2:]-masked_segmentation_relabeled[:-2,:-2]).astype(np.bool)
# d8 = (masked_segmentation_relabeled[2:,:-2]-masked_segmentation_relabeled[:-2,2:]).astype(np.bool)

# neighbors = [set() for _ in range(n_superpixels)]
# edge_coords = defaultdict(set)

# def diff_after_offset(d, y1_offset=0, x1_offset=0, y2_offset=0, x2_offset=0, add_edge_coords=False):
#     ys, xs = np.where(d)
#     y1s = ys + y1_offset
#     x1s = xs + x1_offset
#     y2s = ys + y2_offset
#     x2s = xs + x2_offset
#     for y1,x1,y2,x2 in zip(y1s,x1s,y2s,x2s):
#         l1 = masked_segmentation_relabeled[y1,x1]
#         l2 = masked_segmentation_relabeled[y2,x2]
#         if l1 != -1:
#             neighbors[l1].add(l2)
#             if add_edge_coords:
#                 edge_coords[frozenset([l1,l2])].add((x1,y1))
#                 edge_coords[frozenset([l1,l2])].add((x2,y2))
#         if l2 != -1:
#             neighbors[l2].add(l1)
#             if add_edge_coords:
#                 edge_coords[frozenset([l1,l2])].add((x1,y1))
#                 edge_coords[frozenset([l1,l2])].add((x2,y2))
                
# # cannot work because writing to the same container simultaneous is dangerous
# # fargs = [(d1, 0,0,1,0),(d2, 0,0,0,1),(d3, 0,0,1,1),(d4, 0,1,1,0),
# #          (d5, 0,0,2,0),(d6, 0,0,0,2),(d7, 0,0,2,2),(d8, 0,2,2,0)]
# # Parallel(n_jobs=16)(delayed(f)(t) for t in fargs)
    
# diff_after_offset(d1, 0,0,1,0, add_edge_coords=True)
# diff_after_offset(d2, 0,0,0,1, add_edge_coords=True)
# diff_after_offset(d3, 0,0,1,1, add_edge_coords=True)
# diff_after_offset(d4, 0,1,1,0, add_edge_coords=True)
# diff_after_offset(d5, 0,0,2,0, add_edge_coords=True)
# diff_after_offset(d6, 0,0,0,2, add_edge_coords=True)
# diff_after_offset(d7, 0,0,2,2, add_edge_coords=True)
# diff_after_offset(d8, 0,2,2,0, add_edge_coords=True)

# edge_coords.default_factory = None


# # check symmetry; note that this CANNOT check if neighbors is complete
# A = np.zeros((n_superpixels, n_superpixels))
# for i, nbrs in enumerate(neighbors):
#     q = list([j for j in nbrs if j != -1])
#     A[i, q] = 1    
# assert np.all(A == A.T), 'neighbor list is not symmetric'

# for i in range(n_superpixels):
#     neighbors[i] -= {i}
    
    
# print time.time() - t

    
# # dm.save_pipeline_result(neighbors, 'neighbors', 'pkl')

21.8166899681


In [18]:
# neighbors = dm.load_pipeline_result('neighbors', 'pkl')

In [10]:
t = time.time()

edges = edge_coords.keys()

edge_map = -1 * np.ones_like(dm.image, np.int)
for ei, (e, pts_set) in enumerate(edge_coords.iteritems()):
    xs, ys = zip(*pts_set)
    edge_map[np.array(ys), np.array(xs)] = ei
    
def f(edge_coords):
    return [set(zip(*np.hstack([np.mgrid[max(x-5,0):min(x+5, dm.image_width), 
                                         max(y-5,0):min(y+5, dm.image_height)].reshape((2,-1)) 
                                for x,y in pts]).reshape((2,-1)).tolist())) 
            for e, pts in edge_coords]

b = Parallel(n_jobs=16)(delayed(f)(s) for s in np.array_split(edge_coords.items(), 16))
neighborhood_pts = [p for bb in b for p in bb]

edge_neighbors = defaultdict(set)
for ei, pts in enumerate(neighborhood_pts):
    xs, ys = zip(*pts)
    eis = set(edge_map[ys,xs]) - {-1, ei}
    edge_neighbors[edges[ei]] |= set([edges[ei] for ei in eis])

# dm.save_pipeline_result(edge_neighbors, 'edgeNeighbors','pkl')

print time.time() - t

57.6408178806


In [11]:
edge_coords_sorted = {}
edge_midpoint = {}

for e, pts in edge_coords.iteritems():
    X = np.array(list(pts), dtype=np.float) # n-by-2
    c = X.mean(axis=0)
    edge_midpoint[e] = c
    Xc = X - c
    U,S,V = np.linalg.svd(np.dot(Xc.T, Xc))
    u1 = U[:,0]
    projs = np.dot(Xc,u1)
    order = projs.argsort()
    if Xc[order[0],0] > Xc[order[-1],0]:
        order = order[::-1]
    edge_coords_sorted[e] = X[order].astype(np.int)
    
dm.save_pipeline_result(edge_coords_sorted, 'edgeCoords', 'pkl')
dm.save_pipeline_result(edge_midpoint, 'edgeMidpoints', 'pkl')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_edgeCoords.pkl
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_edgeMidpoints.pkl


In [12]:
dedge_vectors = defaultdict(float)

for e in edge_coords.keys():
    c = edge_midpoint[e]
    i, j = e
    if i == -1:
        vector_ji = sp_properties[j,:2][::-1] - c
    elif j == -1:
        vector_ji = c - sp_properties[i,:2][::-1]
    else:
        vector_ji = sp_properties[i,:2][::-1] - sp_properties[j,:2][::-1]

    dedge_vectors[(i,j)] = vector_ji/np.linalg.norm(vector_ji)
    dedge_vectors[(j,i)] = -dedge_vectors[(i,j)]

dedge_vectors.default_factory = None

dm.save_pipeline_result(dedge_vectors, 'edgeVectors', 'pkl')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_edgeVectors.pkl


In [13]:
dedge_neighbors = defaultdict(set)
for (i,j), es in edge_neighbors.iteritems():

    if len(es) == 0:
        print 'WARNING: edge (%d,%d) has no neighbors'%(i,j)
        ls = []
    else:
        ls = set.union(*[{(a,b),(b,a)} for a,b in es])
    
    dedge_neighbors[(i,j)] |= set((a,b) for a,b in ls 
                                  if not (i==b or j==a) and\
                                  np.dot(dedge_vectors[(i,j)], dedge_vectors[(a,b)]) > -.5) - {(j,i),(i,j)}

    dedge_neighbors[(j,i)] |= set((a,b) for a,b in ls 
                                  if not (j==b or i==a) and\
                                  np.dot(dedge_vectors[(j,i)], dedge_vectors[(a,b)]) > -.5) - {(j,i),(i,j)}

dedge_neighbors.default_factory = None
dm.save_pipeline_result(dedge_neighbors, 'dedgeNeighbors', 'pkl')


border_len_thresh = 20
neighbors_long_border = [set([nbr for nbr in nbrs if len(edge_coords[frozenset([n,nbr])]) > border_len_thresh]) 
                             for n, nbrs in enumerate(neighbors)]
dm.save_pipeline_result(neighbors_long_border, 'neighborsLongBorder', 'pkl')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_dedgeNeighbors.pkl
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_neighborsLongBorder.pkl
